# Variants-Experiment 1.6
## Summed hidden representations and concatenated output logits.
## Mode. Compute classifications of all tiny networks, in all HiCo layers, and take the mode as the final classification.

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join
import cv2
import pandas as pd
import os
import random as rn
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
from keras import backend as K

In [2]:
SEED = 321
rn.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
SUB_REGION_SCALE = 14
NUM_HICO_LAYER = 5
NUM_TN = [10, 8, 6, 4, 2]
NUM_CONNECTION = [0, 5, 4, 3, 2]

#dataset specific parameters
NUM_CLASS = 5

## Data Pre-Processing

In [4]:
with open('../input/hicotine1-layer1-biodiversity/fyp/CIFAR-10.pickle', 'rb') as f:
    X_train_cropped_list, y_train_one_hot, X_val_cropped_list, y_val_one_hot, X_test_cropped_list, y_test_one_hot, coordinate_list, scale_list = pickle.load(f)

## HiCo Layer 1

In [5]:
#build ANN model
ensemble = []
for i in range(NUM_TN[0]):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE*3))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUM_CLASS, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ensemble.append((model, i, coordinate_list[i], scale_list[i]))

In [6]:
for i in range(NUM_TN[0]):
  ensemble[i][0].load_weights('../input/hicotine1-layer1-biodiversity/fyp/CIFAR-10_' + str(i) + '.h5')

In [7]:
hico_layers = []
hico_layers.append(ensemble)

In [8]:
del hico_layers[1:]

## HiCo Layer 2+

In [9]:
# Function to get input of layer i-1
def get_previous_layer_input(hico_layers, layer, tn, train_image):
  input_hr = []
  input_ol = []
    
  if layer == 0:
    previous_layer_input = train_image[tn[1]]
    return previous_layer_input

  elif layer > 0:
    for i in range(NUM_CONNECTION[layer]):
      get_input = (K.function(hico_layers[layer-1][tn[1][i]][0].layers[0].input, hico_layers[layer-1][tn[1][i]][0].layers[1].output))
      input_hr.append(get_input(get_previous_layer_input(hico_layers, layer-1, hico_layers[layer-1][tn[1][i]], train_image)))
      
    input_hr = np.array(input_hr)
    input_hr = np.sum(input_hr, axis=0)

    for i in range(NUM_CONNECTION[layer]):
      get_input = (K.function(hico_layers[layer-1][tn[1][i]][0].layers[0].input, hico_layers[layer-1][tn[1][i]][0].layers[2].output))
      input_ol.append(get_input(get_previous_layer_input(hico_layers, layer-1, hico_layers[layer-1][tn[1][i]], train_image)))

    input_ol = np.array(input_ol)
    input_ol = np.concatenate(input_ol, axis=1)

    input = np.concatenate((input_hr, input_ol), axis=1)
    
    previous_layer_input = input
    return previous_layer_input

In [10]:
for i in range(1, NUM_HICO_LAYER):
  print('Layer %d' %i)
  ensemble = []
  X_train_input_list = []
  X_test_input_list = []

  for j in range(NUM_TN[i]):
    # Build model of HiCo layer i
    connection = tuple(rn.sample(range(len(hico_layers[i-1])), k=NUM_CONNECTION[i]))
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=64+NUM_CLASS*NUM_CONNECTION[i]))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(NUM_CLASS, activation = 'softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    ensemble.append((model, connection))
  hico_layers.append(ensemble)
  print('HICO LENGTH')
  print(len(hico_layers))

  for j in range(NUM_TN[i]):
    # Get train hidden representation from HiCo layer i-1
    X_train_input_hr = []
    X_train_input_ol = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_train_input_hr.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_train_cropped_list)))
    X_train_input_hr = np.array(X_train_input_hr)
    X_train_input_hr = np.sum(X_train_input_hr, axis=0)

    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[2].output))
      X_train_input_ol.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_train_cropped_list)))
    X_train_input_ol = np.array(X_train_input_ol)
    X_train_input_ol = np.concatenate(X_train_input_ol, axis=1)

    X_train_input = np.concatenate((X_train_input_hr, X_train_input_ol), axis=1)
    X_train_input_list.append(X_train_input)

    # Get test hidden representation from HiCo layer i-1
    X_test_input_hr = []
    X_test_input_ol = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_test_input_hr.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input_hr = np.array(X_test_input_hr)
    X_test_input_hr = np.sum(X_test_input_hr, axis=0)

    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[2].output))
      X_test_input_ol.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input_ol = np.array(X_test_input_ol)
    X_test_input_ol = np.concatenate(X_test_input_ol, axis=1)

    X_test_input = np.concatenate((X_test_input_hr, X_test_input_ol), axis=1)
    X_test_input_list.append(X_test_input)

  #train model of HiCo layer i
  for j in range(NUM_TN[i]):
    print('Model %d' %j)
    hico_layers[i][j][0].fit(X_train_input_list[j], y_train_one_hot, validation_data=(X_test_input_list[j], y_test_one_hot), epochs=5, batch_size=128)

Layer 1
HICO LENGTH
2
Model 0
Epoch 1/5
28/28 [==============================] - 1s 21ms/step - loss: 2.0537 - accuracy: 0.2445 - val_loss: 1.4125 - val_accuracy: 0.3533
Epoch 2/5
28/28 [==============================] - 0s 4ms/step - loss: 1.3525 - accuracy: 0.3911 - val_loss: 1.3065 - val_accuracy: 0.4273
Epoch 3/5
28/28 [==============================] - 0s 3ms/step - loss: 1.2967 - accuracy: 0.4403 - val_loss: 1.2956 - val_accuracy: 0.4296
Epoch 4/5
28/28 [==============================] - 0s 3ms/step - loss: 1.2551 - accuracy: 0.4547 - val_loss: 1.2734 - val_accuracy: 0.4503
Epoch 5/5
28/28 [==============================] - 0s 4ms/step - loss: 1.2118 - accuracy: 0.4861 - val_loss: 1.2758 - val_accuracy: 0.4411
Model 1
Epoch 1/5
28/28 [==============================] - 1s 8ms/step - loss: 1.5588 - accuracy: 0.2949 - val_loss: 1.3274 - val_accuracy: 0.3880
Epoch 2/5
28/28 [==============================] - 0s 4ms/step - loss: 1.2912 - accuracy: 0.4106 - val_loss: 1.2852 - val_accur

# Model Evaluation

In [11]:
# Predict on test image
y_pred_list = []
for j in range(NUM_TN[0]):
  y_pred = np.argmax(hico_layers[0][j][0].predict(X_test_cropped_list[j]), axis=1)
  y_pred_list.append(y_pred)

for i in range(1, NUM_HICO_LAYER):
  for j in range(NUM_TN[i]):

    # Get test hidden representation from HiCo layer i-1
    X_test_input_hr = []
    X_test_input_ol = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_test_input_hr.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input_hr = np.array(X_test_input_hr)
    X_test_input_hr = np.sum(X_test_input_hr, axis=0)

    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[2].output))
      X_test_input_ol.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input_ol = np.array(X_test_input_ol)
    X_test_input_ol = np.concatenate(X_test_input_ol, axis=1)

    X_test_input = np.concatenate((X_test_input_hr, X_test_input_ol), axis=1)

    y_pred = np.argmax(hico_layers[i][j][0].predict(X_test_input), axis=1)
    y_pred_list.append(y_pred)

# HiCo voting (mode)
y_pred_list = np.array(y_pred_list)
y_pred_list = np.transpose(y_pred_list, (1, 0))
y_pred_list = stats.mode(y_pred_list, axis=1)[0]
y_pred_list = np.squeeze(y_pred_list)

In [12]:
y_test = np.argmax(y_test_one_hot, axis=1)

In [13]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy

0.45958429561200925